In [ ]:
!pip3 install gitpython

In [2]:
import os
import zipfile
from git import Repo
import pandas as pd
from tensorflow import keras
from keras import layers

In [3]:
if not os.path.exists("./dataset/archive.zip"):
    Repo.clone_from("https://github.com/werrio5/sudoku-dataset", "./dataset")

In [4]:
zf = zipfile.ZipFile("dataset/archive.zip")
df = pd.read_csv(zf.open('sudoku.csv'))
quizzes = df["quizzes"]
solutions = df["solutions"]

In [10]:
# split dataset
def strToNums(str):
  return list(map(lambda chr: int(chr) / 10, list(str)))

train_x = quizzes.sample(frac=0.8, random_state=322).apply(strToNums)
train_y = solutions.loc[solutions.index.isin(train_x.index)].apply(strToNums)
test_x = quizzes.loc[~quizzes.index.isin(train_x.index)].apply(strToNums)
test_y = solutions.loc[solutions.index.isin(test_x.index)].apply(strToNums)

In [ ]:
num_classes = 10 # 0..0.9, 0 = undefined
input_shape = shape=(81)

model = keras.Sequential()
model.add(keras.Input(input_shape))
for i in range(0, 60):
  model.add(layers.Dense(num_classes, activation="sigmoid"))
  if (i % 5 == 0):
    model.add(layers.Dropout(0.15))
model.add(layers.Dense(num_classes, activation=keras.layers.Lambda( lambda f: round(f, 1) )(input_shape)))

model.summary()